In [1]:
import pandas as pd
import numpy as nm
import requests
import json
import mysql
import re
from bs4 import BeautifulSoup


In [2]:
#Constants

start_page = 1
end_page = 10 # For 1000 rows select 11 and for 9943 select 101
imdb_movie_ids_filename = "imdb_movie_ids.txt"
omdb_api_key = '461a254e'


In [3]:
def get_title_id_from_div(div):
    title_id = ""
    try:
        href = div.find_all('a', href=True)[0]['href']
        title_id = href.split('/')[2]
    except:
        print("Raised exception" + str(div))

    return title_id


# Getting list of IMDB Titles and saving in the file
for page_no in range(start_page, end_page):
    print(page_no)
    response = requests.get(url="https://www.imdb.com/list/ls057823854/?page=" + str(page_no))
    soup = BeautifulSoup(response.text, 'html.parser')
    div_header = soup.select('h3.lister-item-header')

    if page_no == start_page:
        with open(imdb_movie_ids_filename, "w+") as file:
            for title in div_header:
                title_id = get_title_id_from_div(title)
                file.write(title_id + "\n")
    else:
        with open(imdb_movie_ids_filename, "a+") as file:
            for title in div_header:
                title_id = get_title_id_from_div(title)
                file.write(title_id + "\n")
    

1
2
3
4
5
6
7
8
9


In [4]:
# Getting Movie Details from the API calls

def convert_to_int(value):
    result = 0
    if value is not None and len(value.strip()) > 0:
        result = re.sub('\D', '', value)
        if len(result) == 0:
            result = 0
    
    return int(result)
   

def get_movie_details(movie_id):
    movie_details ={}
    try:
        response = requests.get("http://www.omdbapi.com/?apikey=" + omdb_api_key + "&i=" + str(movie_id.strip()))
        result = json.loads(response.text)
        movie_details['imdbRating'] = result['imdbRating']
        movie_details['Director'] = result['Director']
        movie_details['Actors'] = result['Actors']
        movie_details['Awards'] = result['Awards']
        movie_details['Genre'] = result['Genre']
        movie_details['Released'] = result['Released']
        movie_details['movie_id'] = movie_id.strip()
        # Add code to find other field values
        
        
        web_scrapping_headers = {'user-agent': 'Mozilla/5.0'}
        web_scrapping_response = requests.get("https://www.imdb.com/title/" + str(movie_id.strip()), web_scrapping_headers)
        soup = BeautifulSoup(web_scrapping_response.content, 'html.parser')

        
 
        #reviews - users and critic
        user_review = 0
        critic_review = 0
        reviews_divs =  soup.findAll("div",{"class":"titleReviewBarItem"})              
        if reviews_divs and len(reviews_divs) >= 2:
            reviews = reviews_divs[1].findAll('a')
        
            if reviews and len(reviews) >= 2:
                user_review = convert_to_int(reviews[0].text.split(" ")[0])
                critic_review = convert_to_int(reviews[1].text.split(" ")[0])

        
        #Popularity
        popularity = ""
        try:
            popularity_divs = soup.findAll("div",{"class":"titleReviewBarSubItem"})
            if popularity_divs and len(popularity_divs) >=3:
                popularity = popularity_divs[2].findAll("span",{"class":"subText"})[0].text
                match = re.search(r'[0-9].*',popularity)
                popularity = convert_to_int(popularity[match.start():match.end()])
        except:
            pass
            
        
        #Budget, Opening weekend USA Sales, Cumulative Worldwide Gross Sales, Gross USA Sales

        divTag = soup.find_all("div", {"id": "titleDetails"})
        budget, weekend_US_sales, gross_US_sales,worldwide_gross_sales  = 0, 0, 0 , 0
        
        
        for tag in divTag:
            divTags = tag.find_all("div", {"class": "txt-block"})
            for tag in (divTags):

                if "Budget:" in tag.text:
                    budget = convert_to_int(re.findall(r'\$[0-9,]*',tag.text)[0])
                if "Opening Weekend USA:" in tag.text:
                    weekend_US_sales = convert_to_int(re.findall(r'\$[0-9,]*',tag.text)[0])
                if "Gross USA:" in tag.text:
                    gross_US_sales = convert_to_int(re.findall(r'\$[0-9,]*',tag.text)[0])
                if "Cumulative Worldwide Gross:" in tag.text:
                    worldwide_gross_sales = convert_to_int(re.findall(r'\$[0-9,]*',tag.text)[0])
            
        movie_details['user_review'] = user_review
        movie_details['critic_review'] = critic_review
        movie_details['popularity'] = popularity
        movie_details['budget'] = budget
        movie_details['weekend_US_sales'] = weekend_US_sales
        movie_details['gross_US_sales'] = gross_US_sales
        movie_details['worldwide_gross_sales'] = worldwide_gross_sales
        return movie_details
    except Exception as e:
        print("Exception in making API calls " + str(movie_id))
        print(e)
        return None




In [5]:
result = []
count = 1
#Reading Files
with open(imdb_movie_ids_filename, "r") as file:
    #Reading the file
    for movie_id in file.readlines():
        print(count)
        count += 1
        if movie_id is None or movie_id.strip() == "":
            continue
        movie_details = get_movie_details(movie_id)
        if movie_details:
            result.append(movie_details)





1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
Exception in making API calls tt1034314

list index out of range
154
155
156
Exception in making API calls tt2099556

list index out of range
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
Exception in making API calls tt0110413

list index out of range
215
216
217
218
219
220
221
222
223
224
225
226
227
228
2

In [6]:
#Creating the dataframe
df = pd.DataFrame(result)
print(df)


                                                Actors  \
0    Tim Roth, Amanda Plummer, Laura Lovelace, John...   
1    Andrew Garfield, Emma Stone, Jamie Foxx, Dane ...   
2    Tim Robbins, Morgan Freeman, Bob Gunton, Willi...   
3    Mark Hamill, Harrison Ford, Carrie Fisher, Pet...   
4    Michael J. Fox, Christopher Lloyd, Lea Thompso...   
5    Emilio Estevez, Paul Gleason, Anthony Michael ...   
6    Sean Astin, Josh Brolin, Jeff Cohen, Corey Fel...   
7    Jodie Foster, Lawrence A. Bonney, Kasi Lemmons...   
8    Sam Neill, Laura Dern, Jeff Goldblum, Richard ...   
9    Rowan Atkinson, Matthew Broderick, Niketa Cala...   
10   Morgan Freeman, Andrew Kevin Walker, Daniel Za...   
11       Tom Hanks, Tim Allen, Don Rickles, Jim Varney   
12   Jeff Bridges, John Goodman, Julianne Moore, St...   
13   Tom Hanks, Tom Sizemore, Edward Burns, Barry P...   
14   Edward Norton, Brad Pitt, Meat Loaf, Zach Grenier   
15   Keanu Reeves, Laurence Fishburne, Carrie-Anne ...   
16   Liam Nees

In [7]:
df.to_csv("movieData_1_10.csv")

In [48]:
import pymongo

In [49]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["imdb"]
mycol = mydb["movies"] #create collection electronics

In [50]:
mycol.insert_many(result)


In [8]:
df.head(10)

,Actors,Awards,Director,Genre,Released,budget,critic_review,gross_US_sales,imdbRating,movie_id,popularity,user_review,weekend_US_sales,worldwide_gross_sales
0,"Tim Roth, Amanda Plummer, Laura Lovelace, John...",Won 1 Oscar. Another 68 wins & 74 nominations.,Quentin Tarantino,"Crime, Drama",14 Oct 1994,8000000,282,107928762,8.9,tt0110912,130,2839,9311882,214179088
1,"Andrew Garfield, Emma Stone, Jamie Foxx, Dane ...",4 wins & 30 nominations.,Marc Webb,"Action, Adventure, Sci-Fi",02 May 2014,200000000,536,202853933,6.6,tt1872181,1552,1162,91608337,708982323
2,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Nominated for 7 Oscars. Another 21 wins & 35 n...,Frank Darabont,Drama,14 Oct 1994,25000000,224,28699976,9.3,tt0111161,66,7361,727327,28786657
3,"Mark Hamill, Harrison Ford, Carrie Fisher, Pet...",Won 6 Oscars. Another 52 wins & 28 nominations.,George Lucas,"Action, Adventure, Fantasy, Sci-Fi",25 May 1977,11000000,181,460998507,8.6,tt0076759,369,1765,1554475,775512064
4,"Michael J. Fox, Christopher Lloyd, Lea Thompso...",Won 1 Oscar. Another 19 wins & 25 nominations.,Robert Zemeckis,"Adventure, Comedy, Sci-Fi",03 Jul 1985,19000000,235,211159762,8.5,tt0088763,117,1078,11152500,388355305
5,"Emilio Estevez, Paul Gleason, Anthony Michael ...",3 wins.,John Hughes,"Comedy, Drama",15 Feb 1985,1000000,151,45875171,7.9,tt0088847,435,837,5107599,51525171
6,"Sean Astin, Josh Brolin, Jeff Cohen, Corey Fel...",3 wins & 6 nominations.,Richard Donner,"Adventure, Comedy, Family",07 Jun 1985,19000000,116,62367596,7.8,tt0089218,267,475,9105913,62613569
7,"Jodie Foster, Lawrence A. Bonney, Kasi Lemmons...",Won 5 Oscars. Another 63 wins & 51 nominations.,Jonathan Demme,"Crime, Drama, Thriller",14 Feb 1991,19000000,213,130742922,8.6,tt0102926,252,1179,13766814,272753884
8,"Sam Neill, Laura Dern, Jeff Goldblum, Richard ...",Won 3 Oscars. Another 39 wins & 26 nominations.,Steven Spielberg,"Action, Adventure, Sci-Fi, Thriller",11 Jun 1993,63000000,355,402828120,8.1,tt0107290,246,1075,47026828,1032020521
9,"Rowan Atkinson, Matthew Broderick, Niketa Cala...",Won 2 Oscars. Another 35 wins & 34 nominations.,"Roger Allers, Rob Minkoff","Animation, Adventure, Drama, Family, Musical",24 Jun 1994,45000000,227,422783777,8.5,tt0110357,329,927,1586753,968511805


# ML

In [145]:
df1 = df.copy()
Actors = df1['Actors'].str.split(",", n = 3, expand = True)
Actors.columns = ['Actor_1', 'Actor_2', 'Actor_3', 'Actor_4']
director = df1['Director'].str.split(",", n = 1, expand = True)
genre = df1['Genre'].str.split(",", n = 1, expand = True)

NameError: name 'pd' is not defined